In [29]:
from langdetect import detect
from langdetect import LangDetectException
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [28]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('vader_lexicon')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to /home/tal012/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/tal012/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/tal012/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/tal012/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [63]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/tal012/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [20]:
import fitz  # PyMuPDF

def extract_text_from_pdf(pdf_path):
    # Open the PDF file
    pdf_document = fitz.open(pdf_path)
    text = ""
    
    # Iterate over each page
    for page_num in range(pdf_document.page_count):
        page = pdf_document.load_page(page_num)
        text += page.get_text()
    
    return text

In [92]:
#textbooks from pdf form:
phili_7 = extract_text_from_pdf('phillipeans/Phili_7.pdf')
phili_8 = extract_text_from_pdf('phillipeans/phili_8.pdf')
phili_9 = extract_text_from_pdf('phillipeans/Phili_9.pdf')
#india_7 = extract_text_from_pdf('india/India_7.pdf')
india_8 = extract_text_from_pdf('india/India_8.pdf')
india_9 = extract_text_from_pdf('india/India_9.pdf')
china_7 = extract_text_from_pdf('china/China_7.pdf')
china_8 = extract_text_from_pdf('china/china_8.pdf')
china_9 = extract_text_from_pdf('china/china_9.pdf')
turkey_9 = extract_text_from_pdf('turkey/Turkey_9.pdf')
tun_8 = extract_text_from_pdf('Tunisia/Tun_8.pdf')
tun_9 = extract_text_from_pdf('Tunisia/Tun_9.pdf')
saudi_7 = extract_text_from_pdf('saudi/Saudi_7.pdf')
saudi_8 = extract_text_from_pdf('saudi/Saudi_8.pdf')
saudi_9 = extract_text_from_pdf('saudi/Saudi_9.pdf')

In [93]:
with open('vietnam/Viet_7.txt','r')as file:
    viet_7 = file.read()
with open('vietnam/Viet_8.txt', 'r', encoding='utf-8', errors='ignore') as file:
    viet_8 = file.read()
with open('mexico/Mexico_7.txt','r', encoding = 'utf-8', errors = 'ignore') as file:
    mexico_7 = file.read()
with open('mexico/Mexico_8.txt','r', encoding = 'utf-8', errors = 'ignore') as file:
    mexico_8 = file.read()

In [94]:
textbooks = [phili_7, phili_8, phili_9, 
             #india_7, 
             india_8, india_9, 
             china_7, china_8, china_9, 
             turkey_9, 
             tun_8, tun_9, 
             saudi_7,saudi_8, saudi_9,
            viet_7, viet_8,
            mexico_7, mexico_8]

In [95]:
def clean_text(post):
    if detect(post) != 'en':
        return ''
    # Make posts lowercase
    post = post.lower()

    # Remove punctuation
    post = re.sub(r'[^a-zA-Z\s]', ' ', post)
    
    # Remove words with repeated letters
    #post = re.sub(r'([a-zA-Z])\1+', r'\1', post)
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(post)
    filtered_text = [word for word in word_tokens if word not in stop_words]
    
    return ' '.join(filtered_text)

In [96]:
cleanTextbooks = []
for i in textbooks:
    cleanTextbooks.append(clean_text(i))

In [97]:
menWords = [" man ", " boy ", " male ", " brother ", " father ", " son ", " husband ", " king ", " prince ", " uncle ", " nephew ", " he ", " him ", " his ", " gentleman ", " sir ", " mr. ", " hero ", " lord ", " patriarch ", " men "]

womenWords = [" woman ", " girl ", " female ", " sister ", " mother ", " daughter ", " wife ", " queen ", " princess ", " aunt ", " niece ", " she ", " her ", " hers ", " lady ", " ma'am "," madam ", " mrs. ", " ms. ", " miss ", " heroine ", " dame ", " matriarch ", " women "]

menWordsNoSpace = ["man", "boy", "male", "brother", "father", "son", "husband", "king", "prince", "uncle", "nephew", "he", "him", "his", "gentleman", "sir", "mr", "hero", "lord", "patriarch", "men"]
womenWordsNoSpace = ["woman", "girl", "female", "sister", "mother", "daughter", "wife", "queen", "princess", "aunt", "niece", "she", "her", "hers", "lady", "ma'am","madam", "mrs", "heroine", "dame", "matriarch", "women"]


In [98]:
#Cleaned the textbooks before getting the gendered words
menPara = [[] for _ in range(18)]
womenPara = [[] for _ in range(18)]
for i in range(0, 18):
    text = cleanTextbooks[i]
    paragraphs = [text[j:j+100] for j in range(0, len(text), 100)]

    for paragraph in paragraphs:
        for j in menWords:
            if j in paragraph.lower():  # Converts paragraph to lowercase for case-insensitive search
                menPara[i].append(paragraph)
        for j in womenWords:
            if j in paragraph.lower():  # Converts paragraph to lowercase for case-insensitive search
                womenPara[i].append(paragraph)

In [99]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [100]:
tobeTFIDF = [" ".join(" ".join(row) for row in menPara)," ".join(" ".join(row) for row in womenPara)]

In [101]:
# Initialize TfidfVectorizer
vectorizer = TfidfVectorizer()

# Fit and transform the data
tfidf_matrix = vectorizer.fit_transform(tobeTFIDF)

# Get feature names (vocabulary)
feature_names = vectorizer.get_feature_names_out()

# Convert the TF-IDF matrix to a dense representation for easy interpretation
dense_tfidf = tfidf_matrix.toarray()

In [102]:
# Get TF-IDF scores for menPara (Document 0) or womenPara (Document 1)
doc_index = 0  # Change to 1 for womenPara
doc_tfidf_scores = dense_tfidf[doc_index]

# Pair each word with its TF-IDF score
word_score_pairs = list(zip(feature_names, doc_tfidf_scores))

# Sort the words by TF-IDF score in descending order
sorted_word_scores = sorted(word_score_pairs, key=lambda x: x[1], reverse=True)

# Get the top 100 words
top_100_words_ms = sorted_word_scores[:100]
top_100_words_m = [word for word, score in sorted_word_scores[:100]]


# Print the top 100 words with their TF-IDF scores
#print(f"Top 100 TF-IDF words for Document {doc_index}:")
#for word, score in top_100_words:
    #print(f"{word}: {score}")

In [109]:
# Get TF-IDF scores for menPara (Document 0) or womenPara (Document 1)
doc_index = 1  # Change to 1 for womenPara
doc_tfidf_scores = dense_tfidf[doc_index]

# Pair each word with its TF-IDF score
word_score_pairs = list(zip(feature_names, doc_tfidf_scores))

# Sort the words by TF-IDF score in descending order
sorted_word_scores = sorted(word_score_pairs, key=lambda x: x[1], reverse=True)

# Get the top 100 words
top_100_words_fs = sorted_word_scores[:100]
top_100_words_f = [word for word, score in sorted_word_scores[:100]]


# Print the top 100 words with their TF-IDF scores
#print(f"Top 100 TF-IDF words for Document {doc_index}:")
#for word, score in top_100_words:
    #print(f"{word}: {score}")

In [104]:
common = list(set(top_100_words_f) & set(top_100_words_m))
len(common)

In [105]:
top_100_words_f = [word for word in top_100_words_f if word not in common]
top_100_words_f = [word for word in top_100_words_f if word not in womenWordsNoSpace]

In [106]:
top_100_words_m = [word for word in top_100_words_m if word not in common]
top_100_words_m = [word for word in top_100_words_m if word not in menWordsNoSpace]

In [107]:
import csv

filename = 'femaleTFIDF'

# Open the file in write mode
with open(filename, mode='w', newline='') as file:
    writer = csv.writer(file)
    
    # Write the header
    writer.writerow(['Word'])
    
    # Write each word in the list as a row
    for word in top_100_words_f:
        writer.writerow([word])

print(f"CSV file '{filename}' has been created.")

CSV file 'femaleTFIDF' has been created.


In [108]:
import csv

filename = 'maleTFIDF'

# Open the file in write mode
with open(filename, mode='w', newline='') as file:
    writer = csv.writer(file)
    
    # Write the header
    writer.writerow(['Word'])
    
    # Write each word in the list as a row
    for word in top_100_words_m:
        writer.writerow([word])

print(f"CSV file '{filename}' has been created.")

CSV file 'maleTFIDF' has been created.
